In [2]:
import os
import mxnet as mx
from mxnet import gluon, nd, ndarray

import pandas as pd
import numpy as np

In [3]:
num_emb = 64
opt = 'Adam'
lr = 0.02
mmntm = 0.
wd = 0.
batch_size = 32
ctx = mx.cpu()

In [4]:
def max_id(fname):
    mu = 0
    mi = 0
    with open(fname) as f:
        for line in f:
            tks = line.strip().split('\t')
            if len(tks) != 4:
                continue
            mu = max(mu, int(tks[0]))
            mi = max(mi, int(tks[1]))
    return mu + 1, mi + 1
max_users, max_items = max_id('./ml-100k/ua.base')
(max_users, max_items)

(944, 1683)

In [5]:
len(np.unique(train_df.values[:,1]))

NameError: name 'train_df' is not defined

In [6]:
len(train_df,unique())

NameError: name 'train_df' is not defined

In [125]:
len(np.unique(train_df['USER_ID'].values))

943

In [ ]:
list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |

In [ ]:
col_names=['ITEM_ID', 'title', 'release_date', 'video_release_date','IMDb URL','unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Noir','Horror','Musical','Mystery','Romance','SciFi','Thriller','War','Western']

In [ ]:
item_info.iloc[:,3:].idxmax(1)

In [147]:
item_info=pd.read_csv('./ml-100k/u.item', sep='|',names=col_names,encoding='latin-1')

In [150]:
item_info=item_info.drop(['video_release_date','IMDb URL'],axis=1)

In [153]:
item_info.to_csv('./data/item_info.csv')

In [139]:
user_info= pd.read_csv('./ml-100k/u.user', sep='|',names=['USER_ID', 'AGE', 'GENDER', 'OCCUPATION','ZIP'])

In [152]:
user_info.to_csv('./data/user_info.csv')

In [156]:
!aws s3 ls s3://product-recommendation-personalize

                           PRE sagemaker/
2019-02-25 13:12:44     982143 DEMO-movie-lens-100k.csv


In [158]:
!aws s3 cp ./data/user_info.csv s3://product-recommendation-personalize/

upload: data/user_info.csv to s3://product-recommendation-personalize/user_info.csv


In [159]:
!aws s3 cp ./data/item_info.csv s3://product-recommendation-personalize/

upload: data/item_info.csv to s3://product-recommendation-personalize/item_info.csv


In [7]:
train_df = pd.read_csv('./ml-100k/ua.base', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
test_df = pd.read_csv('./ml-100k/ua.test', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
train_df

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,1,1,5,874965758
1,1,2,3,876893171
...,...,...,...,...
90568,943,1228,3,888640275
90569,943,1330,3,888692465


In [160]:
train_df.to_csv('./data/ratings.csv')

In [161]:
!aws s3 cp ./data/ratings.csv s3://product-recommendation-personalize/

upload: data/ratings.csv to s3://product-recommendation-personalize/ratings.csv


In [62]:
train_data_iter.__len__()

2831

In [61]:
90570%batch_size

10

In [64]:
train_df.shape[0]//batch_size

2830

In [8]:
train_data = nd.array(train_df[['USER_ID','ITEM_ID']].values, dtype=np.float32)
train_label = nd.array(train_df[['RATING']].values, dtype=np.float32)

test_data = nd.array(test_df[['USER_ID','ITEM_ID']].values, dtype=np.float32)
test_label = nd.array(test_df[['RATING']].values, dtype=np.float32)

In [9]:
class SparseMatrixDataset(gluon.data.Dataset):
    def __init__(self, data, label):
        assert data.shape[0] == len(label)
        self.data = data
        self.label = label
        if isinstance(label, ndarray.NDArray) and len(label.shape) == 1:
            self._label = label.asnumpy()
        else:
            self._label = label       
        
    def __getitem__(self, idx):
        return self.data[idx, 0], self.data[idx, 1], self.label[idx]
    
    def __len__(self):
        return self.data.shape[0]

In [36]:

mx.ndarray.dot(mx.nd.array([1,2,3]),mx.nd.array([1,2,3]))


[ 14.]
<NDArray 1 @cpu(0)>

In [50]:
class MFNet(gluon.Block):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFNet, self).__init__()
        
        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb
        
        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)
            self.dropout = gluon.nn.Dropout(dropout_p)
            
    def forward(self, users, items):
        a = self.user_embeddings(users)
        b = self.item_embeddings(items)
        #predictions=nd.multiply(a, b)
        predictions = mx.ndarray.dot(mx.nd.array(a),mx.nd.array(b))     
        return predictions


In [54]:
net=MFNet(max_users, max_items, 32)
net.initialize(mx.init.Xavier(magnitude=2))
net.collect_params()
#net(mx.nd.array([0]),mx.nd.array([1]))

mfnet23_ (
  Parameter mfnet23_embedding0_weight (shape=(944, 32), dtype=float32)
  Parameter mfnet23_embedding1_weight (shape=(1683, 32), dtype=float32)
)

In [37]:
class MFNet(gluon.HybridBlock):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFNet, self).__init__()
        
        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb
        
        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)
            self.dropout = gluon.nn.Dropout(dropout_p)
            
    def hybrid_forward(self, F, users, items):
        a = self.user_embeddings(users)
        b = self.item_embeddings(items)
        #predictions=nd.multiply(a, b)
        predictions = mx.ndarray.dot(mx.nd.array(a),mx.nd.array(b))     
        return predictions



In [42]:
net=MFNet(max_users, max_items, 32)
net.initialize(mx.init.Xavier(magnitude=2))

net.hybridize() 



#net.forward(mx.nd.array([0]),mx.nd.array([1]))
#net.feature_net.export('test')


In [46]:
mx.nd.array([0])


[ 0.]
<NDArray 1 @cpu(0)>

In [49]:
net(mx.nd.array([0]),mx.nd.array([0]))

TypeError: source_array must be array like object

In [26]:
def eval_net(data, net):
    acc = mx.metric.RMSE()
    for i, (user, item, label) in enumerate(data):
        user = user.as_in_context(ctx).reshape((len(user),))
        item = item.as_in_context(ctx).reshape((len(user),))
        label = label.as_in_context(ctx).reshape((len(user),))
        predictions = net(user, item)
        loss_function = gluon.loss.L2Loss()
        loss = loss_function(predictions, label)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]


In [164]:

def train(train,test, users,items,epochs, lr, emb_size):
    a = []
    b = []
    
    net = MFNet(max_users=users, max_items=items, num_emb=emb_size, dropout_p=0.)
    net.collect_params()
    loss_function = gluon.loss.L2Loss()

    net.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)
    net.hybridize()
    trainer = gluon.Trainer(net.collect_params(), 'sgd',{'learning_rate': lr, 'momentum': 0.9})

    for e in range(epochs):
        print("epoch: {}".format(e))
        for i, (user, item, label) in enumerate(train):
            user = user.as_in_context(ctx).reshape((len(user),))
            item = item.as_in_context(ctx).reshape((len(user),))
            label = label.as_in_context(ctx).reshape((len(user),))
            with mx.autograd.record():
                output = net(user, item)               
                loss = loss_function(output, label)
                loss.backward()
            net.collect_params().values()
            trainer.step(len(user))
        a = eval_net(test, net)
        b = eval_net(train, net)
        print("EPOCH {}: RMSE ON TRAINING and TEST: {}. {}".format(e,a,b))
        
    return net

In [12]:
train_data_iter = gluon.data.DataLoader(SparseMatrixDataset(train_data, train_label), 
                                        shuffle=True, batch_size=batch_size)
test_data_iter = gluon.data.DataLoader(SparseMatrixDataset(test_data, test_label),
                                          shuffle=True, batch_size=batch_size)

In [27]:
lr=0.01
epochs=1
emb=64
net = MFNet(max_users, max_items, emb, dropout_p=0.)
net.collect_params()
loss_function = gluon.loss.L2Loss()

net.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)
net.hybridize()
net(mx.nd.array([0]),mx.nd.array([1]))
net.export('model', epoch=epochs)
trainer = gluon.Trainer(net.collect_params(), 'sgd',{'learning_rate': lr, 'momentum': 0.9})


TypeError: type <class 'mxnet.symbol.symbol.Symbol'> not supported

In [ ]:

for e in range(epochs):
    print("epoch: {}".format(e))
    for i, (user, item, label) in enumerate(train_data_iter):
        user = user.as_in_context(ctx).reshape((len(user),))
        item = item.as_in_context(ctx).reshape((len(user),))
        label = label.as_in_context(ctx).reshape((len(user),))
        with mx.autograd.record():
            output = net(user, item)               
            loss = loss_function(output, label)
            loss.backward()
        net.collect_params().values()
        trainer.step(len(user))
    a = eval_net(test_data_iter, net)
    b = eval_net(train_data_iter, net)
    print("EPOCH {}: RMSE ON TRAINING and TEST: {}. {}".format(e,a,b))

In [172]:
model.hybridize()
model.forward(mx.nd.array([0]),mx.nd.array([0]))
model.export('model', epoch=epochs)

RuntimeError: Please first call block.hybridize() and then run forward with this block at least once before calling export.

In [11]:
net = MFBlock(max_users=max_users, max_items=max_items, num_emb=num_emb, dropout_p=0.)
net.collect_params()

mfblock0_ (
  Parameter mfblock0_embedding0_weight (shape=(944, 64), dtype=float32)
  Parameter mfblock0_embedding1_weight (shape=(1683, 64), dtype=float32)
)

In [17]:
loss_function = gluon.loss.L2Loss()

In [12]:
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)

In [103]:
trainer = gluon.Trainer(net.collect_params(), 'sgd',
                        {'learning_rate': lr, 'wd': wd, 'momentum': 0.9})

In [104]:
train_data_iter = gluon.data.DataLoader(SparseMatrixDataset(train_data, train_label), 
                                        shuffle=True, batch_size=batch_size)
test_data_iter = gluon.data.DataLoader(SparseMatrixDataset(test_data, test_label),
                                          shuffle=True, batch_size=batch_size)

In [106]:
def eval_net(data, net):
    acc = mx.metric.RMSE()
    for i, (user, item, label) in enumerate(data):
        user = user.as_in_context(ctx).reshape((len(user),))
        item = item.as_in_context(ctx).reshape((len(user),))
        label = label.as_in_context(ctx).reshape((len(user),))
        predictions = net(user, item)
        loss = loss_function(predictions, label)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [72]:
eval_net(train_data_iter, net)

3.6948865924526633

In [108]:
epochs=20
def train(data_iter, net):
    a = []
    b = []
    c = []
    d = []
    for e in range(epochs):
        print("epoch: {}".format(e))
        for i, (user, item, label) in enumerate(train_data_iter):
            user = user.as_in_context(ctx).reshape((len(user),))
            item = item.as_in_context(ctx).reshape((len(user),))
            label = label.as_in_context(ctx).reshape((len(user),))
            with mx.autograd.record():
                output = net(user, item)               
                loss = loss_function(output, label)
                loss.backward()
            net.collect_params().values()
            trainer.step(batch_size)
        a = eval_net(test_data_iter, net)
        b = eval_net(train_data_iter, net)
        print("EPOCH {}: RMSE ON TRAINING and TEST: {}. {}".format(e,a,b))
        
    return a, b


In [109]:
(a,b) = train(train_data_iter, net)

epoch: 0
EPOCH 0: RMSE ON TRAINING and TEST: 1.8750390804420083. 1.482538243247106
epoch: 1
EPOCH 1: RMSE ON TRAINING and TEST: 1.6958517009929075. 1.1061352421899686
epoch: 2
EPOCH 2: RMSE ON TRAINING and TEST: 1.6080985059172421. 0.939340492370313
epoch: 3
EPOCH 3: RMSE ON TRAINING and TEST: 1.566502850338564. 0.8455567052953027
epoch: 4
EPOCH 4: RMSE ON TRAINING and TEST: 1.5416291931928214. 0.7880754086479456
epoch: 5
EPOCH 5: RMSE ON TRAINING and TEST: 1.533353523076591. 0.7470146141596289
epoch: 6
EPOCH 6: RMSE ON TRAINING and TEST: 1.5280011777150428. 0.7156961210643138
epoch: 7
EPOCH 7: RMSE ON TRAINING and TEST: 1.5209427374904438. 0.6947740343340459
epoch: 8
EPOCH 8: RMSE ON TRAINING and TEST: 1.5243068394014392. 0.6744094318991539
epoch: 9
EPOCH 9: RMSE ON TRAINING and TEST: 1.521259038933253. 0.6582971533062347
epoch: 10
EPOCH 10: RMSE ON TRAINING and TEST: 1.5240529745312061. 0.6430876020280265
epoch: 11
EPOCH 11: RMSE ON TRAINING and TEST: 1.525570483530982. 0.63117952811

In [86]:
ratings = nd.dot(net.user_embeddings.weight.data(ctx=ctx), net.item_embeddings.weight.data(ctx=ctx).T).asnumpy()
ratings.shape

(944, 1683)

In [93]:
net.forward(mx.nd.array([0]),mx.nd.array([1]))


[ 0.08441807]
<NDArray 1 @cpu(0)>

In [90]:
ratings[0,1]

0.084418066